In [1]:
# COLAB
# from google.colab import drive
# drive.mount('/content/drive')

# # Move segmentation files to dir

# from VesselLabPics_PreTrained import FCN_NetModel as FCN
# from VesselLabPics_PreTrained import CategoryDictionary as CatDic

#### Convert a raw video to segmented video + mask frames

In [4]:
import os
import torch
import numpy as np
import cv2
import FCN_NetModel as FCN # The net Class
import CategoryDictionary as CatDic

# In function
def fluid_segmentation (InputVideo,
                        UseGPU=True,
                        FreezeBatchNormStatistics=False,
                        Trained_model_path=r"logs//TrainedModelWeiht1m_steps_Semantic_TrainedWithLabPicsAndCOCO_AllSets.torch"):

    OutVideoMain=InputVideo[:-4]+"_Filled.avi"
    MaskVideoMain=InputVideo[:-4]+"_Masks.avi"

    Net=FCN.Net(CatDic.CatNum)
    if UseGPU==True:
        print("USING GPU")
        Net.load_state_dict(torch.load(Trained_model_path))
    else:
        print("USING CPU")
        Net.load_state_dict(torch.load(Trained_model_path, map_location=torch.device('cpu')))
    
    cap = cv2.VideoCapture(InputVideo)
    fourcc = cv2.VideoWriter_fourcc(*"MJPG")
    MainVideoWriter=None
    MaskVideoWriter=None
    MaskMain = []

    while (cap.isOpened()):
        ret, Im = cap.read()
        if ret == False:
            break
        
        h,w,d = Im.shape
        r = np.max([h,w])

        if r > 840:
            fr = 840/r
            Im = cv2.resize(Im,(int(w*fr),int(h*fr)))
        h, w, d = Im.shape
        Imgs = np.expand_dims(Im,axis=0)
        
        if not (type(Im) is np.ndarray):
            continue

        #................................Make Prediction.............................................................................................................
        with torch.autograd.no_grad():
            OutProbDict,OutLbDict=Net.forward(Images=Imgs,TrainMode=False,UseGPU=UseGPU, FreezeBatchNormStatistics=FreezeBatchNormStatistics)
        #................................Make Prediction.............................................................................................................

        my=2
        mx=2
        OutMain = np.zeros(Im.shape, np.uint8) # 475 840 3
        nm = "Filled"

        y = 0
        x = 0
        OutMain[:h,:w]=Im
        
        Lb = OutLbDict[nm].data.cpu().numpy()[0].astype(np.uint8) # mask
        if Lb.mean()<0.001:
            continue

        if nm=='Ignore':
            continue

        # DISPLAYING SEGMENTED VIDEO ("Filled")
        ImOverlay1 = Im.copy()
        ImOverlay1[:, :, 1][Lb==1] = 0
        ImOverlay1[:, :, 0][Lb==1] = 255
        # font = cv2.FONT_HERSHEY_SIMPLEX
        # cv2.putText(ImOverlay1, nm, ( int(w/3), int(h/6)), font, 2, (0, 255, 0), 2, cv2.LINE_AA)
        OutMain[h*y:h*(y+1), w*x:w*(x+1)] = ImOverlay1
        x+=1
        if x>=mx:
            x=0
            y+=1

        # cv2.imshow('Main Classes', OutMain)
        # cv2.waitKey(5)

        # SAVE "Filled"
        if MainVideoWriter is None:
            h, w, d = OutMain.shape
            MainVideoWriter = cv2.VideoWriter(OutVideoMain, fourcc, 30.0, (w, h))
        MainVideoWriter.write(OutMain)

        #-------------------------------------------------------------------------------------------------------------------------
        # FOR MASK VIDEO
        MaskMain.append (Lb)

    # SAVE Mask video
    MaskStack = np.stack(MaskMain)
    fourcc = cv2.VideoWriter_fourcc(*"XVID") # grayscale
    # h,w,d = OutMain.shape
    if MaskVideoWriter is None:
        MaskVideoWriter = cv2.VideoWriter (MaskVideoMain, fourcc, 30.0, (w,h), isColor=False)

    for i, mask in enumerate (MaskStack):
        mask = (mask * 255).astype(np.uint8)
        MaskVideoWriter.write (mask)

    #-----------------------------------------------------------------------------------------------------------------------------
    print("Finished")
    MainVideoWriter.release()
    MaskVideoWriter.release()
    cap.release()
    cv2.destroyAllWindows()
    return









In [5]:
InputVideo=r"C:/Users/jhp98/OneDrive/Desktop/Viscosity Autoencoder/fortest.mp4"
Trained_model_path =r"logs/trained_2023.torch"

fluid_segmentation (InputVideo, Trained_model_path=Trained_model_path)

c:\Users\jhp98\anaconda3\envs\DL\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\jhp98\anaconda3\envs\DL\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


USING GPU
Finished


In [ ]:
from os.path import join

cropped_videos_path = "/content/VesselLabPics_PreTrained/cropped"
cropped_videos_filenames = os.listdir (cropped_videos_path)
cropped_videos_dir = [join (cropped_videos_path, cropped_videos_filenames[i]) for i in range(len(cropped_videos_filenames))]

for i, video in enumerate(cropped_videos_dir):
  fluid_segmentation (video, Trained_model_path=r"/content/VesselLabPics_PreTrained/logs/TrainedModelWeiht1m_steps_Semantic_TrainedWithLabPicsAndCOCO_AllSets.torch")
  print (f"{cropped_videos_filenames[i]} segmented and saved")

USING GPU
Finished
w9.mp4 segmented and saved
USING GPU
Finished
m4.mp4 segmented and saved
USING GPU
Finished
w8.mp4 segmented and saved
USING GPU
Finished
m3.mp4 segmented and saved
USING GPU
Finished
m10.mp4 segmented and saved
USING GPU
Finished
m8.mp4 segmented and saved
USING GPU
Finished
w2.mp4 segmented and saved
USING GPU
Finished
w4.mp4 segmented and saved
USING GPU
Finished
m9.mp4 segmented and saved
USING GPU
Finished
w1.mp4 segmented and saved
USING GPU
Finished
w7.mp4 segmented and saved
USING GPU
Finished
w3.mp4 segmented and saved
USING GPU
Finished
m5.mp4 segmented and saved
USING GPU
Finished
m2.mp4 segmented and saved
USING GPU
Finished
m7.mp4 segmented and saved
USING GPU
Finished
m1.mp4 segmented and saved
USING GPU
Finished
w5.mp4 segmented and saved
USING GPU
Finished
w10.mp4 segmented and saved
USING GPU
Finished
w6.mp4 segmented and saved
USING GPU
Finished
m6.mp4 segmented and saved


In [ ]:
# moving results
all_filenames = os.listdir (cropped_videos_path)
all_dir = [join (cropped_videos_path, all_filenames[i]) for i,vid in enumerate(all_filenames) if vid.endswith(".avi")]
move_dir = join("/content/VesselLabPics_PreTrained/out_dir", f.split("/")[-1])

for f in all_dir:
  move_dir = join("/content/VesselLabPics_PreTrained/out_dir", f.split("/")[-1])
  os.rename(f, move_dir)


# saving results
saved_dir = "/content/VesselLabPics_PreTrained/out_dir"
to_download = os.listdir(saved_dir)
print (f"file to be moved to Drive: \n{to_download}")

for i, vid in enumerate (to_download):
  !cp /content/VesselLabPics_PreTrained/out_dir/{vid} "/content/drive/MyDrive/oscil_segnpz"
  print (f"{vid} saved to Drive")